# Barrier clip

### Add column to colonies GeoDataFrame showing if there is the presence of a barrier or not.

Outline of how to implement this:
* Import Delhi/NDMC/JJC dataset **[DONE]**
* Import barrier clip shapefiles **[DONE]**
* Reproject shapefiles to the same CRS **[DONE]**
* Check validity - turn into a function (that can be used for other shapefiles)
    * Make sure there are no duplicate rows **[DONE]**
    * Make sure there are no duplicate geometries **[DONE]**
    * Make sure that no row has None in geometry field. **[DONE]**
    * Check that barrier clip shapefiles are all (poly)lines. **[DONE]**
    * Make sure there are no validity problems. **[DONE]**
* Use Spatial Join to create identifier column `barrier`. **[DONE]**
    * Create function to do this
    * `barrier` column should be boolean (True or False)
    * Create intermediate columns 'canal', 'railway', and 'drain'.
    * Make 'barrier' as intersection of canal, railway and drain
* Export colonies shapefile. **[DONE]**
* Open in QGIS and manually inspect features.

In [61]:
import os
from importlib import reload
import pandas as pd
import geopandas as gpd
import shapely
import spatial_index_utils

In [194]:
reload(spatial_index_utils)

<module 'spatial_index_utils' from 'C:\\Users\\bwbel\\Google Drive\\slum_project\\spatial_index_python\\spatial_index_utils.py'>

### Import Delhi/NDMC/JJC dataset

In [5]:
colonies = gpd.read_file('delhi_ndmc_jjc_corrected.shp')

In [7]:
colonies.head(2)

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101..."
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100..."


### Import barrier shapefiles

In [13]:
barrier_clip_dir = 'Barrier_Clip'

canal_filepath = os.path.join(barrier_clip_dir, 'Canal', 'Canal.shp')
drain_filepath = os.path.join(barrier_clip_dir, 'Drain', 'Major_Drain.shp')
railway_filepath = os.path.join(barrier_clip_dir, 'Railway', 'Railway_Line.shp')

# Check filepath validity before importing shapefiles
print('canal filepath exists:', os.path.exists(canal_filepath))
print('drain filepath exists:', os.path.exists(drain_filepath))
print('railway filepath exists:', os.path.exists(railway_filepath))

canal filepath exists: True
drain filepath exists: True
railway filepath exists: True


In [14]:
canal = gpd.read_file(canal_filepath)
drain = gpd.read_file(drain_filepath)
railway = gpd.read_file(railway_filepath)

In [15]:
canal.head(2)

,FID,CAN_NM,CAN_CLSF,EL_GND,DIST_NM,geometry
0,0,WESTERN YAMUNA CANAL,None,216.682,None,"LINESTRING (77.15632 28.70021, 77.15519 28.70205)"
1,1,WESTERN YAMUNA CANAL,None,221.601,None,"LINESTRING (77.15714 28.69886, 77.15632 28.70021)"


In [16]:
drain.head(2)

,FID,Drain_type,Drain_Name,MAINTAINED,AC_NAME,DISTRICT,geometry
0,0,None,Asola Drain,None,CHHATARPUR,SOUTH,"LINESTRING (77.19468 28.44483, 77.19467 28.445..."
1,1,None,Asola Drain,None,CHHATARPUR,SOUTH,"MULTILINESTRING ((77.18090 28.45571, 77.18086 ..."


In [17]:
railway.head(2)

,FID,RL_ZONE,geometry
0,0,NORTHERN RAILWAY,"LINESTRING (77.25315 28.63192, 77.25308 28.631..."
1,1,NORTHERN RAILWAY,"LINESTRING (77.25309 28.63194, 77.25302 28.63181)"


### Reproject shapefiles to EPSG:3857

In [18]:
colonies = spatial_index_utils.reproject_gdf?

Signature: spatial_index_utils.reproject_gdf(gdf, epsg_code)
Docstring:
Reprojects GeoDataFrame to CRS with EPSG code

Assigns WKT format of projection to EPSG code to GeoDataFrame.

Args:
gdf: GeoDataFrame with any geometry (e.g., Point, Line, Polygon)
epsg_code: EPSG code (integer)

Returns:
    GeoDataFrame reprojected to new crs (based on EPSG code).
File:      c:\users\bwbel\google drive\slum_project\spatial_index_python\spatial_index_utils.py
Type:      function


In [19]:
colonies = spatial_index_utils.reproject_gdf(gdf=colonies, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [20]:
canal = spatial_index_utils.reproject_gdf(gdf=canal, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [21]:
drain = spatial_index_utils.reproject_gdf(gdf=drain, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [22]:
railway = spatial_index_utils.reproject_gdf(gdf=railway, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


### Check validity of shapefiles (and correct)

#### First, check for duplicate rows

In [30]:
gdf_list = {'colonies': colonies, 'canal': canal, 'drain': drain, 'railway': railway}

In [33]:
for gdf in gdf_list:
    print(gdf, 'has duplicate rows:', spatial_index_utils.gdf_has_duplicate_rows(gdf_list[gdf]))

colonies has duplicate rows: False
canal has duplicate rows: False
drain has duplicate rows: False
railway has duplicate rows: False


#### Second, correct any duplicate geometries

In [36]:
len(colonies)

2811

In [37]:
colonies = spatial_index_utils.remove_duplicate_geom(colonies)

In [38]:
len(colonies) # after removing duplicate geometries

2808

In [39]:
len(canal)

45

In [40]:
canal = spatial_index_utils.remove_duplicate_geom(canal)

In [41]:
len(canal)

45

In [42]:
len(drain)

616

In [43]:
drain = spatial_index_utils.remove_duplicate_geom(drain)

In [44]:
len(drain)

616

In [45]:
len(railway)

5414

In [46]:
railway = spatial_index_utils.remove_duplicate_geom(railway)

In [47]:
len(railway)

5414

#### Third, make sure no row has None in Geometry Field

In [48]:
colonies[colonies['geometry'] == None]

,index,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry


In [49]:
canal[canal['geometry'] == None]

,index,FID,CAN_NM,CAN_CLSF,EL_GND,DIST_NM,geometry


In [50]:
drain[drain['geometry'] == None]

,index,FID,Drain_type,Drain_Name,MAINTAINED,AC_NAME,DISTRICT,geometry


In [51]:
railway[railway['geometry'] == None]

,index,FID,RL_ZONE,geometry


#### Fourth, check that barrier clip shapefiles are all (poly)lines.

In [57]:
geom_type = "Line"
for gdf in gdf_list:
    print(gdf, 'has all geometries of type', geom_type, ":",
          spatial_index_utils.check_geometries(gdf_list[gdf], geom_type))

colonies has all geometries of type Line : False
canal has all geometries of type Line : True
drain has all geometries of type Line : True
railway has all geometries of type Line : True


#### Fifth, ensure no validity problems
* Canal has problems with rows 17 and 18.
* Railway has problems in multiple places

In [58]:
canal.head()

,index,FID,CAN_NM,CAN_CLSF,EL_GND,DIST_NM,geometry
0,0,0,WESTERN YAMUNA CANAL,None,216.682,None,"LINESTRING (8589002.799 3337544.234, 8588876.3..."
1,1,1,WESTERN YAMUNA CANAL,None,221.601,None,"LINESTRING (8589093.770 3337372.930, 8589002.7..."
2,2,2,WESTERN YAMUNA CANAL,None,216.329,None,"LINESTRING (8589093.770 3337372.930, 8589195.6..."
3,3,3,WESTERN YAMUNA CANAL,None,215.459,None,"LINESTRING (8590447.715 3334557.954, 8590314.4..."
4,4,4,WESTERN YAMUNA CANAL,None,216.210,None,"LINESTRING (8589584.266 3336275.195, 8589600.1..."


In [76]:
def print_invalid_rows(gdf):
    """Print rows with invalid geometries"""
    for i, row in gdf.iterrows():
        if not row['geometry'].is_valid:
            print('not valid index', i, '\n', row)
        

In [86]:
# print_invalid_rows(canal) # 2 rows of invalid data

In [78]:
print_invalid_rows(colonies)

In [79]:
print_invalid_rows(drain)

In [85]:
#print_invalid_rows(railway) #there were many rows

#### Test again with recorrected shapefiles (canal and railway)

In [82]:
canal = gpd.read_file(canal_filepath)
canal = spatial_index_utils.reproject_gdf(gdf=canal, epsg_code=3857)
print_invalid_rows(canal)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [83]:
railway = gpd.read_file(railway_filepath)
railway = spatial_index_utils.reproject_gdf(gdf=railway, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [84]:
print_invalid_rows(railway)

### Spatial Join for Intersections

let's start with colonies and canal

In [199]:
colonies = spatial_index_utils.barrier_intersection(colonies, canal, "canal")

In [202]:
colonies = spatial_index_utils.barrier_intersection(colonies, railway, "railway")

In [220]:
colonies = spatial_index_utils.barrier_intersection(colonies, drain, "drain")

In [221]:
colonies.head()

,index,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain
0,0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...",False,False,True
1,1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...",False,True,False
2,2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...",False,True,False
3,3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...",False,True,False
4,4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...",False,True,False


In [224]:
# Create barrier column as being intersection with canal, railway or drain
colonies['barrier'] = colonies['canal'] | colonies['railway'] | colonies["drain"]

In [229]:
colonies.head(10)

,index,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier
0,0,Singhola,3058,H,RV,"POLYGON Z ((8587300.847 3355178.518 0.000, 858...",False,False,True,True
1,1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((8580725.093 3357134.173 0.000, 858...",False,True,False,True
2,2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((8581345.143 3353980.079 0.000, 858...",False,True,False,True
3,3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((8580819.492 3356801.814 0.000, 858...",False,True,False,True
4,4,Kureni,2082,H,RV,"POLYGON Z ((8582448.764 3356971.996 0.000, 858...",False,True,False,True
5,5,Mamoor Pur,2183,H,RV,"POLYGON Z ((8584563.612 3358228.541 0.000, 858...",False,False,True,True
6,6,Guru Ravi Dass Nagar Harijan Basti,1632,G,UV,"POLYGON Z ((8584563.612 3358228.541 0.000, 858...",False,False,False,False
7,7,Baba Hari Dass Nagar(Tikri Khurd),1148,G,RUAC,"POLYGON Z ((8583516.652 3354259.887 0.000, 858...",False,False,True,True
8,8,Sanjay Colony Narela,2869,G,UAC1,"POLYGON Z ((8581528.709 3357162.771 0.000, 858...",False,False,False,False
9,9,Tikri Khurd,3169,H,RV,"POLYGON Z ((8585406.523 3354793.344 0.000, 858...",False,False,True,True


In [230]:
colonies.tail(10)

,index,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier
2798,2801,Tool Room Training Center Wazirpur Industrial ...,4061,None,JJC2,MULTIPOLYGON Z (((8590004.633 3337386.648 0.00...,False,False,False,False
2799,2802,Ambedker Colony Andheria More,4062,None,JJC2,"POLYGON Z ((8591351.569 3313070.398 0.000, 859...",False,False,False,False
2800,2803,Manav Kalyan Camp Giri Nagar Kalkaji,4063,None,JJC2,"POLYGON Z ((8600986.284 3317539.466 0.000, 860...",False,False,False,False
2801,2804,X-Block Nariana Loha Mandi,4064,None,JJC2,"POLYGON Z ((8587320.465 3329597.175 0.000, 858...",False,False,False,False
2802,2805,X-Block Mangol Puri,4065,None,JJC2,"POLYGON Z ((8580598.207 3337039.238 0.000, 858...",False,False,False,False
2803,2806,Shaheed Arjun Dass Camp on Kushak Nalla Betwee...,4066,None,JJC2,"POLYGON Z ((8593433.440 3325349.337 0.000, 859...",False,False,False,False
2804,2807,"Sant Ravi Dass Camp L-Block, Shakur Pur.",4067,None,JJC2,"POLYGON Z ((8587319.108 3336213.658 0.000, 858...",False,False,False,False
2805,2808,Block-17 & 21 Kalyan Puri,4068,None,JJC2,"POLYGON Z ((8606328.421 3326900.546 0.000, 860...",False,False,False,False
2806,2809,Jhuggies Kholi Wala Baba Mandir Rajokari,4069,None,JJC2,"POLYGON Z ((8585316.940 3313858.375 0.000, 858...",False,False,False,False
2807,2810,JJC Janta Jeevan Camp in the back Lane of X-38...,4070,None,JJC2,"POLYGON Z ((8602311.287 3316781.436 0.000, 860...",False,False,False,False


### Save and export files

In [231]:
colonies.to_file("colonies_with_barrier_3Aug2020.shp")

In [232]:
with open("colonies_with_barrier_3Aug2020.data", "wb") as f:
    pickle.dump(colonies, f)